In [1]:
from osgeo import gdal, ogr, os

# The classification result is displayed as a raster. 
# For a better representation the result is transformed into vectors.
outpoly = "D:/cbr/VP2/VP2_SoilMapping/polygone.shp"
naip_fn = 'D:/cbr/VP2/VP2_SoilMapping/Processed/classified_Wetzwil_mitvierKlassen.tif'

sourceRaster = gdal.Open(naip_fn)
band = sourceRaster.GetRasterBand(1)
bandArray = band.ReadAsArray()
driver = ogr.GetDriverByName("ESRI Shapefile")
if os.path.exists(outpoly):
    driver.DeleteDataSource(outpoly)
outDatasource = driver.CreateDataSource(outpoly)
outLayer = outDatasource.CreateLayer("polygonized", srs=None)
newField = ogr.FieldDefn('MYFLD')
outLayer.CreateField(newField)
# Transformation
print("Starting Transformation Raster to Vector")
gdal.Polygonize(band, None, outLayer, 0, [], callback=None )
outDatasource.Destroy()
sourceRaster = None
print("Done")


In [2]:
# Old code - not longe in use
#%reset
#import rasterio
#from rasterio.features import shapes
#import geopandas as gp
#from shapely.geometry import Point
#
#outpoly = "D:/cbr/VP2/VP2_SoilMapping/polygone.shp"
#naip_fn = 'D:/cbr/VP2/VP2_SoilMapping/Processed/classified_Wetzwil_mitvierKlassen.tif'
#
#mask = None
#with rasterio.Env():
#    with rasterio.open(naip_fn) as src:
#        image = src.read(1) # first band
#        results = (
#        {'properties': {'raster_val': v}, 'geometry': s}
#        for i, (s, v) 
#        in enumerate(
#            shapes(image, mask=mask, transform=src.transform)))
#
#
#geoms = list(results)
# # first feature
##print(geoms[0])
#
#gpd_polygonized_raster  = gp.GeoDataFrame.from_features(geoms)
#print(gpd_polygonized_raster)
#
##gpd_polygonized_raster['geometry'] = gpd_polygonized_raster.apply(lambda x: Point((float(x.lon), float(x.lat))), axis=1)
#gpd_polygonized_raster.to_file(outpoly, driver='ESRI Shapefile')

Nothing done.


KeyboardInterrupt: 